<a href="https://colab.research.google.com/github/Gise05624/Proyecto_DataScience_I/blob/main/Entrega_3/ProyectoParteIII_Bargas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final

### Archivo: ProyectoDS_ParteI_Bargas.ipynb
_______________________________________

# ✔ Modelado y Validación

Este proyecto forma parte del análisis comercial de un conjunto de datos con información mensual sobre ventas, costos operativos y satisfacción del cliente, segmentado por región y tipo de producto. En esta tercera parte busco aplicar técnicas de selección de variables y modelado predictivo para estimar el volumen de ventas.

_______________________________________

## 🗂️📈 Preparación de los datos

Se cargan los datos, se codifican las variables categóricas y se convierten las fechas en variables numéricas para su uso en modelos.



In [2]:

import pandas as pd

df = pd.read_csv('ventas_dataset_actdeclase.csv')
df['Mes'] = pd.to_datetime(df['Mes'])
df_modelo = df.copy()
df_modelo = pd.get_dummies(df_modelo, columns=['Region', 'Producto'], drop_first=True)
df_modelo['Mes'] = df_modelo['Mes'].dt.month
df_modelo.head()


,Mes,Ventas,Satisfaccion_Cliente,Costo_Operativo,Region_Norte,Region_Oeste,Region_Sur,Producto_Producto_B,Producto_Producto_C
0,1,4174,4.593086,1794,True,False,False,False,False
1,2,2130,4.559382,2138,True,False,False,False,False
2,3,3169,3.311989,830,True,False,False,False,False
3,4,2482,4.732352,630,True,False,False,False,False
4,5,2685,3.112823,2891,True,False,False,False,False


____________________

## 📊 Selección de variables (Feature Selection)

Se usa el método **SelectKBest** con función `f_regression` para seleccionar las 5 variables más relevantes para predecir las ventas.

In [3]:

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split

X = df_modelo.drop('Ventas', axis=1)
y = df_modelo['Ventas']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

selector = SelectKBest(score_func=f_regression, k=5)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

selected_features = X_train.columns[selector.get_support()].tolist()
print("Variables seleccionadas:", selected_features)


Variables seleccionadas: ['Satisfaccion_Cliente', 'Costo_Operativo', 'Region_Norte', 'Region_Oeste', 'Producto_Producto_C']


__________________________


## 👩🏻‍💻 Entrenamiento del modelo

Se utiliza un modelo de regresión basado en Random Forest para predecir el volumen de ventas.


In [4]:

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model.fit(X_train_selected, y_train)

y_pred = model.predict(X_test_selected)


____________________

## 🧐 Evaluación del modelo

Se evalúa el modelo utilizando métricas de regresión: **MAE**, **RMSE** y **R²**.


In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

MAE: 1048.7724137931034
RMSE: 1177.6051911664065
R²: -0.15432475616027408


___________________

## 📈💰📊 Modelo de Clasificación
Creo una variable categórica que indica si las ventas fueron altas (por encima de la mediana) o bajas. Luego se entrena un modelo `RandomForestClassifier` para predecir esta clase.

In [6]:
df_modelo['Ventas_Altas'] = (df_modelo['Ventas'] > df_modelo['Ventas'].median()).astype(int)
X_clf = df_modelo.drop(['Ventas', 'Ventas_Altas'], axis=1)
y_clf = df_modelo['Ventas_Altas']

from sklearn.model_selection import train_test_split
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)


In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_clf, y_train_clf)
y_pred_clf = clf.predict(X_test_clf)


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("Accuracy:", accuracy_score(y_test_clf, y_pred_clf))
print("Precision:", precision_score(y_test_clf, y_pred_clf))
print("Recall:", recall_score(y_test_clf, y_pred_clf))
print("F1 Score:", f1_score(y_test_clf, y_pred_clf))
print("\nClassification Report:\n", classification_report(y_test_clf, y_pred_clf))


Accuracy: 0.5862068965517241
Precision: 0.625
Recall: 0.35714285714285715
F1 Score: 0.45454545454545453

Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.80      0.67        15
           1       0.62      0.36      0.45        14

    accuracy                           0.59        29
   macro avg       0.60      0.58      0.56        29
weighted avg       0.60      0.59      0.56        29



## 🔍 Conclusiones

En este proyecto trabajé dos enfoques: regresión para predecir ventas y clasificación para identificar si fueron altas o bajas.

En el modelo de regresión, las variables más relevantes fueron la satisfacción del cliente, el costo operativo y algunas regiones y productos. El error promedio fue de MAE 1048.77 y el RMSE de 1177.61, lo que muestra que el modelo no tuvo un buen desempeño, pero sí me permitió detectar que variables internas como la satisfacción del cliente, los costos operativos y ciertos productos o regiones están influyendo en los resultados.

En el modelo de clasificación, la precisión fue de 0.62, con un F1-score de 0.45. Si bien no fueron valores altos, fue útil para empezar a identificar combinaciones de variables asociadas a ventas por encima del promedio.